In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Sun/ML Shock/Final project/

/content/drive/MyDrive/Sun/ML Shock/Final project


In [3]:
# path = '../'
# data_dir = path +'data/'

path = './' #'/content/drive/MyDrive/Sun/ML Shock/Final project/'
data_dir = path +'preprocessing/'


data_type = 'eeg_raw'

In [4]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle

In [5]:
all_file_list = os.listdir(data_dir)
# file_list = [f for f in all_file_list if f.endswith('.npy') and data_type in f]
# label_list = [f for f in all_file_list if f.endswith('.npy') and 'y_' in f]


In [6]:
len(all_file_list)
# file_list[:10], label_list[:10]

28949

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, data_type, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        all_file_list = os.listdir(data_dir)
        self.file_list = [f for f in all_file_list if f.endswith('.npy') and data_type in f]
        self.label_list = [f for f in all_file_list if f.endswith('.npy') and 'y_' in f]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.file_list[idx])
        label_path = os.path.join(self.data_dir, self.label_list[idx])
        data = np.load(file_path)
        class_probs = np.load(label_path)
        label = np.argmax(class_probs)
        if self.transform:
            data = self.transform(data)
        return data, label, class_probs


In [8]:
# load the data
training_data = CustomDataset(data_dir, data_type = data_type)


In [15]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [16]:
# # get a batch of data
# data, label, class_probs = next(iter(train_dataloader))

In [17]:
count = 0
for data, label, class_probs in train_dataloader:
    print(data.shape, data[0,:5,0], label[:5])
    count+=1
    if count>2:
      break

torch.Size([64, 9800, 20]) tensor([-4.8700, -4.5000, -4.5200, -6.3100, -7.2800], dtype=torch.float64) tensor([0, 4, 2, 3, 1, 1, 2, 4, 1, 4, 1, 1, 4, 2, 4, 2, 3, 5, 1, 5, 1, 3, 1, 5,
        0, 4, 3, 1, 3, 2, 1, 1, 4, 1, 4, 5, 2, 5, 4, 4, 3, 0, 1, 2, 5, 3, 5, 4,
        0, 0, 3, 1, 3, 4, 5, 3, 1, 2, 4, 2, 1, 1, 0, 5])
torch.Size([64, 9800, 20]) tensor([  7.7400,  -6.4200,  -5.6300,  -1.1200, -10.9700], dtype=torch.float64) tensor([1, 0, 0, 4, 0, 1, 5, 5, 0, 1, 5, 0, 1, 4, 1, 2, 2, 5, 0, 5, 4, 5, 2, 2,
        0, 5, 2, 3, 4, 3, 1, 4, 4, 5, 1, 0, 5, 1, 3, 1, 0, 5, 5, 0, 3, 1, 3, 0,
        1, 4, 5, 5, 4, 4, 5, 1, 3, 5, 0, 2, 4, 0, 5, 5])


KeyboardInterrupt: 

In [24]:
# load the scaler
with open(path + f'{data_type}_standard_scaler.pkl', 'rb') as f:
    standard_scaler = pickle.load(f)

with open(path + f'{data_type}_min_max_scaler.pkl', 'rb') as f:
    min_max_scaler = pickle.load(f)

In [22]:
# create a data scaler
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

# use the partial function to create a scaler
standard_scaler = StandardScaler()
min_max_scaler = MaxAbsScaler()
for data, _, _ in train_dataloader:
  try:
    scale = standard_scaler.scale_
    # min = min_max_scaler.data_min_
    # max = min_max_scaler.data_max_
  except:
    scale = None
  standard_scaler.partial_fit(data.reshape(-1, data.shape[-1]))
  min_max_scaler.partial_fit(data.reshape(-1, data.shape[-1]))
  scale_new = standard_scaler.scale_
  # min_new = min_max_scaler.data_min_
  # max_new = min_max_scaler.data_max_
  if scale is not None:
    change = np.abs(scale_new - scale)
    # change_minmax = np.abs([min_new - min, max_new - max])
    print(change.max()) #, change_minmax.max())





737.0522408149636
322.1965292621992
450.65868372341447
172.5790170194291
121.25858811167154
149.6966788322884
603.2875650249596
253.5199172801838
77.4145449711898
72.09032447341224
374.4317015442996
44.3679721040952
74.51607187858463
90.03962337643452
45.69836222927347
333.26425376467
154.43367465015604
1839.0415793655789
99.57701109085201
60.94225674042639
128.80391949800833
85.33927377677082
35.79395540646442
31.995668178479264
64.51252873213843
827.1123975063688
77.81787728602376
55.69507914067162
72.24463659461617
47.53236887610274
48.87439308024932
54.733431349633975
45.919696051524625
51.35368739048408
57.47882760857101
50.66947155410071
50.31667894296652
68.13597341525758
44.62983619176566
36.530151793578625
37.41695395720353
21.89954257288946
35.63274503316052
23.530928902159303
39.68430209173266
28.94828552089939
56.04291669903944
36.075053648761696
35.49032806169407
40.222770574265724
54.358393335655364
59.06542928497936
32.1933718721084
61.102726546587974
27.67832991397472
3

KeyboardInterrupt: 

In [23]:
# save the scaler

with open(path +f'{data_type}_standard_scaler.pkl', 'wb') as f:
    pickle.dump(standard_scaler, f)
with open(path + f'{data_type}_min_max_scaler.pkl', 'wb') as f:
    pickle.dump(min_max_scaler, f)

In [ ]:
# # load the scaler
# with open(path + f'{data_type}_standard_scaler.pkl', 'rb') as f:
#     standard_scaler = pickle.load(f)

# with open(path + f'{data_type}_min_max_scaler.pkl', 'rb') as f:
#     min_max_scaler = pickle.load(f)


In [ ]:
# # scale the data
# for data, _, _ in train_dataloader:
#     print(data.shape, data[0,:5,0])
#     orig_shape = data.shape
#     data = scaler.transform(data.reshape(-1, data.shape[-1])).reshape(orig_shape)
#     print(data.shape, data[0,:5,0])

In [ ]:

# training_data_scaled = CustomDataset(data_dir, data_type = data_type, transform = scaler.transform)
# train_dataloader_scaled = DataLoader(training_data_scaled, batch_size=4, shuffle=False)

In [ ]:
# for data, label, class_probs in train_dataloader_scaled:
#     print(data.shape, data[0,:5,0], label)